#Агент Plan-and-execute
Агенты, работающие по принципу "планируй и выполняй", достигают цели, сначала планируя, что делать, а затем выполняя подзадачи.
Планирование почти всегда осуществляется LLM.
Выполнение обычно осуществляется отдельным агентом (оснащенным инструментами).

#Imports

In [ ]:
!pip install langchain_experimental


In [ ]:
! pip install langchain openai google-search-results
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain.chains import LLMMathChain

[ChatOpenAI](https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html)

[OpenAI](https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html)

[SerpAPIWrapper](https://api.python.langchain.com/en/latest/utilities/langchain.utilities.serpapi.SerpAPIWrapper.html)

[Tool](https://api.python.langchain.com/en/latest/tools/langchain.tools.base.Tool.html)

[LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html)

#TOOLS

Для получения ключа API от SerpAPI выполните следующие шаги:

1. **Регистрация на SerpAPI**:
   - Перейдите на [официальный сайт SerpAPI](https://serpapi.com/).
   - Зарегистрируйтесь на сайте.

2. **Получение ключа API**:
   - После регистрации и входа в систему, перейдите в раздел аккаунта или настройки профиля.
   - В этом разделе должен быть предоставлен ваш уникальный ключ API. Обычно он выглядит как длинная строка символов.

3. **Использование ключа API**:
   - Когда у вас есть ключ, вы можете использовать его в вашем коде, передавая его при инициализации `SerpAPIWrapper` или установив его как переменную окружения, как я показал ранее.

**Примечание**: Обратите внимание, что SerpAPI может предоставить вам бесплатный период на пробное использование, но после истечения этого периода вам, возможно, потребуется оплатить услуги для продолжения использования API.


In [3]:
import ipywidgets as widgets
from IPython.display import display
import os

def handle_submit(sender):
    print("Ключ Serp_API установлен!")

# Создайте поле для ввода пароля (ключа API для SerpAPI)
serpapi_key_input = widgets.Password(
    placeholder='Введите ваш ключ API от SerpAPI здесь',
    description='SerpAPI ключ:',
    disabled=False
)
serpapi_key_input.on_submit(handle_submit)
display(serpapi_key_input)


Password(description='SerpAPI ключ:', placeholder='Введите ваш ключ API от SerpAPI здесь')

Ключ Serp_API установлен!


In [4]:
os.environ['SERPAPI_API_KEY'] = serpapi_key_input.value

In [5]:
def on_button_click(b):
    os.environ['OPENAI_API_KEY'] = key_input.value
    print("Ключ OpenAI_API установлен!")

key_input = widgets.Password(
    placeholder='Введите ваш ключ API здесь',
    description='API ключ:',
    disabled=False
)
apply_button = widgets.Button(description="Применить")
apply_button.on_click(on_button_click)
display(key_input, apply_button)

Password(description='API ключ:', placeholder='Введите ваш ключ API здесь')

Button(description='Применить', style=ButtonStyle())

Ключ OpenAI_API установлен!


In [6]:
openai_api_key = key_input.value
os.environ['OPENAI_API_KEY'] = openai_api_key

In [7]:
from langchain.utilities import SerpAPIWrapper

In [8]:
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="полезен при необходимости ответить на вопросы о текущих событиях"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="полезен, когда нужно ответить на вопросы по математике"
    ),
]

#Planner, Executor, and Agent

In [9]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

#Пример выполнения

In [10]:
agent.run("Кто является девушкой Лео Ди Каприо? Выведи число равное ее нынешнему возрасту в годах, возведенному в степень 0,43?")



> Entering new PlanAndExecute chain...
steps=[Step(value='Research who is currently in a relationship with Leonardo DiCaprio.'), Step(value='Find out the age of the person identified in step 1.'), Step(value="Calculate the person's age raised to the power of 0.43."), Step(value='Output the result obtained in step 3.'), Step(value="Given the above steps taken, respond to the user's original question. \n\n")]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search",
  "action_input": "Leonardo DiCaprio current relationship"
}
```
Observation: ['Who is Leonardo DiCaprio currently dating? Leonardo DiCaprio has most recently been linked to 25-year-old Italian model Vittoria Ceretti. The ...']
Thought:I have searched for information on Leonardo DiCaprio's current relationship. According to my search results, Leonardo DiCaprio has most recently been linked to 25-year-old Italian model Vittoria Ceretti.

> Finished chain.
*****

Step: Research who is currently in a relations

'According to my research, Leonardo DiCaprio is currently linked to 25-year-old Italian model Vittoria Ceretti. The age of Vittoria Ceretti raised to the power of 0.43 is approximately 7.0801.'

    
    
    > Entering new PlanAndExecute chain...
    steps=[Step(value="Search for Leo DiCaprio's girlfriend on the internet."), Step(value='Find her current age.'), Step(value='Raise her current age to the 0.43 power using a calculator or programming language.'), Step(value='Output the result.'), Step(value="Given the above steps taken, respond to the user's original question.\n\n")]
    
    > Entering new AgentExecutor chain...
    Action:
    ```
    {
      "action": "Search",
      "action_input": "Who is Leo DiCaprio's girlfriend?"
    }
    ```
    
    
    Observation: DiCaprio broke up with girlfriend Camila Morrone, 25, in the summer of 2022, after dating for four years. He's since been linked to another famous supermodel – Gigi Hadid. The power couple were first supposedly an item in September after being spotted getting cozy during a party at New York Fashion Week.
    Thought:Based on the previous observation, I can provide the answer to the current objective.
    Action:
    ```
    {
      "action": "Final Answer",
      "action_input": "Leo DiCaprio is currently linked to Gigi Hadid."
    }
    ```
    
    
    > Finished chain.
    *****
    
    Step: Search for Leo DiCaprio's girlfriend on the internet.
    
    Response: Leo DiCaprio is currently linked to Gigi Hadid.
    
    > Entering new AgentExecutor chain...
    Action:
    ```
    {
      "action": "Search",
      "action_input": "What is Gigi Hadid's current age?"
    }
    ```
    
    Observation: 28 years
    Thought:Previous steps: steps=[(Step(value="Search for Leo DiCaprio's girlfriend on the internet."), StepResponse(response='Leo DiCaprio is currently linked to Gigi Hadid.'))]
    
    Current objective: value='Find her current age.'
    
    Action:
    ```
    {
      "action": "Search",
      "action_input": "What is Gigi Hadid's current age?"
    }
    ```
    
    
    Observation: 28 years
    Thought:Previous steps: steps=[(Step(value="Search for Leo DiCaprio's girlfriend on the internet."), StepResponse(response='Leo DiCaprio is currently linked to Gigi Hadid.')), (Step(value='Find her current age.'), StepResponse(response='28 years'))]
    
    Current objective: None
    
    Action:
    ```
    {
      "action": "Final Answer",
      "action_input": "Gigi Hadid's current age is 28 years."
    }
    ```
    
    
    
    > Finished chain.
    *****
    
    Step: Find her current age.
    
    Response: Gigi Hadid's current age is 28 years.
    
    > Entering new AgentExecutor chain...
    Action:
    ```
    {
      "action": "Calculator",
      "action_input": "28 ** 0.43"
    }
    ```
    
    
    > Entering new LLMMathChain chain...
    28 ** 0.43
    ```text
    28 ** 0.43
    ```
    ...numexpr.evaluate("28 ** 0.43")...
    
    Answer: 4.1906168361987195
    > Finished chain.
    
    Observation: Answer: 4.1906168361987195
    Thought:The next step is to provide the answer to the user's question.
    
    Action:
    ```
    {
      "action": "Final Answer",
      "action_input": "Gigi Hadid's current age raised to the 0.43 power is approximately 4.19."
    }
    ```
    
    
    
    > Finished chain.
    *****
    
    Step: Raise her current age to the 0.43 power using a calculator or programming language.
    
    Response: Gigi Hadid's current age raised to the 0.43 power is approximately 4.19.
    
    > Entering new AgentExecutor chain...
    Action:
    ```
    {
      "action": "Final Answer",
      "action_input": "The result is approximately 4.19."
    }
    ```
    
    
    > Finished chain.
    *****
    
    Step: Output the result.
    
    Response: The result is approximately 4.19.
    
    > Entering new AgentExecutor chain...
    Action:
    ```
    {
      "action": "Final Answer",
      "action_input": "Gigi Hadid's current age raised to the 0.43 power is approximately 4.19."
    }
    ```
    
    
    > Finished chain.
    *****
    
    Step: Given the above steps taken, respond to the user's original question.
    
    
    
    Response: Gigi Hadid's current age raised to the 0.43 power is approximately 4.19.
    > Finished chain.





    "Gigi Hadid's current age raised to the 0.43 power is approximately 4.19."